In [2]:
import zipfile
import os

# Define paths
zip_path = "/content/dataset.zip"
extract_path = "/content/extracted_files"

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extracted to: {extract_path}")

Extracted to: /content/extracted_files


## Extracting Files

In [4]:
with_mask_dir = '/content/extracted_files/dataset/with_mask'
without_mask_dir = '/content/extracted_files/dataset/without_mask'

In [15]:
pip install -r requirements.txt

# SVM

In [22]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV , RandomizedSearchCV
from scipy.stats import randint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall, AUC

In [10]:
# Parameters for LBP
radius = 1
n_points = 8 * radius
METHOD = 'uniform'

# Initialize lists
features = []
labels = []

# Function to extract HOG features
def extract_hog_features(image):
    """
    Extract Histogram of Oriented Gradients (HOG) features from an image.
    """
    # Resize image
    image = cv2.resize(image, (128, 128))
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Extract HOG features
    hog_features = hog(gray_image, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    return hog_features

# Function to extract LBP features
def extract_lbp_features(image, radius=1, n_points=8, method='uniform'):
    """
    Extract Local Binary Pattern (LBP) features from an image.
    """
    # Resize image
    image = cv2.resize(image, (128, 128))
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Extract LBP features
    lbp = local_binary_pattern(gray_image, n_points, radius, method)
    # Compute histogram
    (hist, _) = np.histogram(lbp.ravel(),
                             bins=np.arange(0, n_points + 3),
                             range=(0, n_points + 2))
    # Normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

# Load images and extract features
for category, label in [(with_mask_dir, 1), (without_mask_dir, 0)]:
    for filename in os.listdir(category):
        img_path = os.path.join(category, filename)
        image = cv2.imread(img_path)
        if image is not None:
            hog_features = extract_hog_features(image)
            lbp_features = extract_lbp_features(image, radius, n_points, METHOD)
            # Concatenate HOG and LBP features
            combined_features = np.hstack((hog_features, lbp_features))
            features.append(combined_features)
            labels.append(label)

# Convert to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Standardize features
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

# Initialize SVM classifier
svm = SVC()

# Define parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],       # Regularization parameter
    'kernel': ['linear', 'rbf']   # Kernel type
}

# Initialize GridSearchCV
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Train classifier with grid search
grid_search.fit(X_train, y_train)

# Best parameters from grid search
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Predict and evaluate with the best estimator
best_svm = grid_search.best_estimator_
train_predictions = best_svm.predict(X_train)
test_predictions = best_svm.predict(X_test)

train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

print(f'Training Accuracy: {train_accuracy * 100:.2f}%')
print(f'Testing Accuracy: {test_accuracy * 100:.2f}%')


Best Parameters: {'C': 10, 'kernel': 'rbf'}
Training Accuracy: 100.00%
Testing Accuracy: 94.63%


In [12]:
# Generate classification report
print("Classification Report:")
print(classification_report(y_test, test_predictions))

# Generate confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, test_predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       386
           1       0.94      0.96      0.95       433

    accuracy                           0.95       819
   macro avg       0.95      0.95      0.95       819
weighted avg       0.95      0.95      0.95       819

Confusion Matrix:
[[359  27]
 [ 17 416]]


## Random Forest Classifier

In [13]:
# Initialize Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Define parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(10, 100),
    'max_depth': randint(2, 50),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 11)
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                   n_iter=100, cv=3, scoring='accuracy', n_jobs=-1, random_state=42)

# Train classifier with random search
random_search.fit(X_train, y_train)

# Best parameters from random search
best_params = random_search.best_params_
print(f'Best Parameters: {best_params}')

# Predict and evaluate with the best estimator
best_rf = random_search.best_estimator_
train_predictions = best_rf.predict(X_train)
test_predictions = best_rf.predict(X_test)

train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

print(f'Training Accuracy: {train_accuracy * 100:.2f}%')
print(f'Testing Accuracy: {test_accuracy * 100:.2f}%')


Best Parameters: {'max_depth': 27, 'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 93}
Training Accuracy: 99.85%
Testing Accuracy: 91.21%


In [14]:
# Generate classification report
print("Classification Report:")
print(classification_report(y_test, test_predictions))

# Generate confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, test_predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90       386
           1       0.88      0.97      0.92       433

    accuracy                           0.91       819
   macro avg       0.92      0.91      0.91       819
weighted avg       0.92      0.91      0.91       819

Confusion Matrix:
[[329  57]
 [ 15 418]]


## ANN

In [25]:
# Convert labels to categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Define ANN model
def create_ann_model():
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(2, activation='softmax')  # Binary classification
    ])
    model.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy', Precision(), Recall(), AUC()]
  )
    return model

# Create and train the model
model = create_ann_model()
history = model.fit(X_train, y_train_cat, epochs=20, batch_size=32, validation_data=(X_test, y_test_cat), verbose=1)

# Evaluate model on test data
test_results = model.evaluate(X_test, y_test_cat, verbose=1)

# Unpack results based on the number of metrics
test_loss = test_results[0]  # Loss is always the first value
test_accuracy = test_results[1]  # Accuracy (second value)
test_precision = test_results[2]  # Precision
test_recall = test_results[3]  # Recall
test_auc = test_results[4]  # AUC

# Print results
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Precision: {test_precision * 100:.2f}%')
print(f'Test Recall: {test_recall * 100:.2f}%')
print(f'Test AUC: {test_auc:.4f}')



Epoch 1/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.7646 - auc_2: 0.8417 - loss: 0.5023 - precision_2: 0.7646 - recall_2: 0.7646 - val_accuracy: 0.9182 - val_auc_2: 0.9773 - val_loss: 0.1973 - val_precision_2: 0.9182 - val_recall_2: 0.9182
Epoch 2/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9176 - auc_2: 0.9754 - loss: 0.2002 - precision_2: 0.9176 - recall_2: 0.9176 - val_accuracy: 0.9292 - val_auc_2: 0.9787 - val_loss: 0.1886 - val_precision_2: 0.9292 - val_recall_2: 0.9292
Epoch 3/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9465 - auc_2: 0.9880 - loss: 0.1381 - precision_2: 0.9465 - recall_2: 0.9465 - val_accuracy: 0.9463 - val_auc_2: 0.9867 - val_loss: 0.1513 - val_precision_2: 0.9463 - val_recall_2: 0.9463
Epoch 4/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9674 - auc_2: 0.9945 - loss: 0.0896 - precision_2: 0.9674 - recall_2: 0.9674 - val_accuracy: 0.9475 - val_auc_2: 0.9857 - val_loss: 0.1585 - val_precision_2: 0.9475 - v